# 1. Instalación e importación de bibliotecas

In [1]:
!git clone https://github.com/angelfergar/ChurnRetention-SP.git

Cloning into 'ChurnRetention-SP'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 48 (delta 15), reused 20 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 2.55 MiB | 9.18 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn.metrics import precision_recall_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [3]:
dataset_path = '/content/ChurnRetention-SP/data/preprocessed_customerChurn.csv'
df = pd.read_csv(dataset_path)

# 2. Equilibrado de datos

Creamos los dataset que usaremos para el entrenamiento de los modelos

In [4]:
df_X = df.drop('Churn Label', axis=1)
df_y = df['Churn Label']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=0)

Aumentamos de forma sintética la clase minoritaria un 65% y reduce la clase mayoritaria en un 25% (Se llega a estas cifras a base de prueba y error). De esta forma, pasamos de tener una proporción de 3:1 a una de 1.25 sin introducir un exceso de ruido que pueda provocar overfitting en el modelo ni perder una gran cantidad de datos.

In [6]:
smote = SMOTE(sampling_strategy=0.6)
under = RandomUnderSampler(sampling_strategy=0.8)

# Creamos un pipeline para aplicar ambas técnicas a la vez
pipeline = Pipeline([('smote', smote), ('under', under)])

# Obtenemos dos nuevos datasets con los datos equilibrados
X_train_smote, y_train_smote = pipeline.fit_resample(X_train, y_train)

print('Distribución de clases antes de Balanceo:', y_train.value_counts())
print('Distribución de clases después de Balanceo:', pd.Series(y_train_smote).value_counts())

Distribución de clases antes de Balanceo: Churn Label
0    3615
1    1315
Name: count, dtype: int64
Distribución de clases después de Balanceo: Churn Label
0    2711
1    2169
Name: count, dtype: int64


Este ajuste no supone una reducción excesiva ya que se han eliminado 904 muestras de la clase mayoritaria. De la misma forma, el haber añadido, de forma sintética, 854 muestras a la clase minoritaria no debería crear mucho ruido a la hora de desarrollar los modelos, por lo que no tendría que haber problemas de overfitting.

Este equilibrado se utilizará como punto de partida para optimizar los
diferentes modelos, pero se usarán una serie de valores adicionales a los elegidos cuando apliquemos las técnicas de validación cruzada y grid search sobre el modelo que mejores métricas haya obtenido.



# 3. Entrenamiento de los modelos

En este apartado vamos a trabajar con cinco modelos de machine learning: **Linear Regression, Decision Tree, Random Forest, XGBoost y LightGBM.** El objetivo es entrenar los modelos y analizar las métricas que son más relevantes en el caso tratado.

Al contar con un cierto desbalance en el dataset, métricas como Precision y Recall pueden no ser suficientes para evaluar el rendimiento de los modelos. Por una parte, la precisión podría verse afectada por el incremento de falsos positivos, dado que SMOTE ha generado casos sintéticos que podrían aumentar las predicciones de los clientes que se han dado de baja. Por otra parte, el Recall podría aumentar por el mayor énfasis dado a la clase minoritaria, pero esto no significa que se pueda garantizar que las predicciones sean consistentes o útiles en el contexto trabajado.

Debido a estas razones, **usaremos F1-Score como métrica principal** para determinar la eficacia de los modelos. Esto se debe a que combina Precision y Recall, penalizando modelos que tengan un desequilibrio entre ambas. Esto es particularmente importante tras haber aplicado las técnicas de equilibrado de datos, ya que el F1-Score permite verificar si el modelo realmente ha aprendido a identificar de manera efectiva los casos de clientes que se han dado de baja.

También **usaremos otras métricas como la ROC Curve y la Precision-Recall Curve**. Es cierto que la ROC Curve ya que el dataset tratado cuenta con un cierto rango de desbalanceo, y esta métrica tiende a obtener una mayoría de verdaderos negativos, lo que puede hacer que esta métrica se vea un poco inflada. Por ello, también analizaremos la Precisión-Recall Curve, ya que esta es mucho más óptima en contextos donde existan desequilibrios entre las clases, ya que refleja cual es el rendimiento de la clase minoritaria.

